In [26]:
# Import the requests library.
import requests
from citipy import citipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime


# Import the API key.
from config import api_key

In [27]:
beaches_df = pd.read_csv("Resources/beaches_data.csv")
beaches_df.head()

,Beach_ID,Beach_Name,Latitude,Longitude,Sunrise,Sunset,Weather_Description,Temperature,Water_Temperature,Cloud_Cover,Wind_Speed,Wave_Height
0,0,Imperial Beach,32.575831,-117.126577,07:07 AM,05:57 PM,Patchy rain possible,62,67,73,12,3.0
1,1,Border Field State Park,32.544940,-117.122341,07:07 AM,05:57 PM,Patchy rain possible,62,67,70,12,3.0
2,2,Pelican State Beach,41.992395,-124.209664,07:52 AM,06:09 PM,Light rain shower,48,52,100,4,10.5
3,3,Crescent Beach,41.728363,-124.153178,07:51 AM,06:10 PM,Light rain shower,46,51,100,6,7.9
4,4,Redwood National Park,41.208970,-124.119056,07:49 AM,06:11 PM,Patchy rain possible,47,52,86,7,9.5


In [28]:
# Renamed "Unnamed: 0" column and set it as the index
beaches_df = beaches_df.rename(columns={'Unnamed: 0': ''})
beaches_df = beaches_df.set_index('Beach_ID')
beaches_df.head()

,Beach_Name,Latitude,Longitude,Sunrise,Sunset,Weather_Description,Temperature,Water_Temperature,Cloud_Cover,Wind_Speed,Wave_Height
Beach_ID,,,,,,,,,,,
0,Imperial Beach,32.575831,-117.126577,07:07 AM,05:57 PM,Patchy rain possible,62,67,73,12,3.0
1,Border Field State Park,32.544940,-117.122341,07:07 AM,05:57 PM,Patchy rain possible,62,67,70,12,3.0
2,Pelican State Beach,41.992395,-124.209664,07:52 AM,06:09 PM,Light rain shower,48,52,100,4,10.5
3,Crescent Beach,41.728363,-124.153178,07:51 AM,06:10 PM,Light rain shower,46,51,100,6,7.9
4,Redwood National Park,41.208970,-124.119056,07:49 AM,06:11 PM,Patchy rain possible,47,52,86,7,9.5


In [29]:
beaches_df.dtypes

Beach_Name              object
Latitude               float64
Longitude              float64
Sunrise                 object
Sunset                  object
Weather_Description     object
Temperature              int64
Water_Temperature        int64
Cloud_Cover              int64
Wind_Speed               int64
Wave_Height            float64
dtype: object

In [30]:
# Starting URL for Weather Map API Call.
url = 'http://api.worldweatheronline.com/premium/v1/marine.ashx?key=' + api_key 
print(url)

http://api.worldweatheronline.com/premium/v1/marine.ashx?key=9242feb85e11428d81f20803220111


In [31]:
# Create a set of latitude and longitude combinations.
beach = beaches_df['Beach_Name']
lats = beaches_df['Latitude']
lngs = beaches_df['Longitude']

beaches_lat_lngs = zip(beach, lats, lngs)
coordinates = list(beaches_lat_lngs)

for coordinate in coordinates:
    print(coordinate[0], coordinate[1], coordinate[2])

Imperial Beach 32.57583119 -117.1265769
Border Field State Park 32.54494004 -117.1223413
Pelican State Beach 41.992395 -124.209664
Crescent Beach 41.72836258 -124.1531784
Redwood National Park 41.20897003 -124.1190559
Prairie Creek Redwoods State Beach 41.40174089 -124.0416494
Humboldt Lagoons State Park 41.22522304 -124.0980863
Sue-meg State Park 41.13665293 -124.155297
Trinidad State Beach 41.06582175 -124.1497893
Little River State Beach 41.01688007 -124.1117529
Clam Beach County Park 40.99847401 -124.1151808
Sinkyone Wilderness State Park 39.92043479 -123.9397125
Westport-Union Landing State Beach 39.67716307 -123.7912683
MacKerricher State Park 39.4886489 -123.7881132
Caspar Headlands State Beach 39.3627382 -123.8172252
Russian Gulch State Park 39.33310956 -123.8058978
Manchester State Park 38.98385949 -123.6993187
Schooner Gulch State Beach 38.86821949 -123.6547063
Glass Beach 39.45277476 -123.8133215
Gualala Point Regional Park 38.76076189 -123.526723
Kruse Rhododendron State Re

In [32]:
print(coordinates[209])

('San Onofre State Beach', 33.37388206, -117.5662879)


In [33]:
len(beach)

249

In [34]:
# test lat and long variables which are going to be beach coordinates
## using Grandview Coordinates= 33.07705649, -117.3110183
lat = 33.51038632
long = -117.7525398

# Create an endpoint URL for a beach. 

beach_url = url + "&format=json&q=" + str(lat) + "," + str(long)

#get it in text format
beach_weather = requests.get(beach_url)

if beach_weather.status_code == 200:
    print (f"Beach Weather found.")

else: 
    print (f"Beach weather not found.")

Beach Weather found.


In [35]:
# Create an empty list to hold the weather data.
beach_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, j in enumerate(coordinates) :

    # Group beaches in sets of 30 for logging purposes.
    if (i % 30 == 0 and i >= 30):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    beach_url = url + "&format=json&q=" + str(coordinates[i-1][1]) + "," + str(coordinates[i-1][2])

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} --- {coordinates[i-1][0]}")
    
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        beach_weather = requests.get(beach_url).json()
        
        # Parse out the needed data.
        sunrise = beach_weather['data']['weather'][0]['astronomy'][0]['sunrise']
        sunset = beach_weather['data']['weather'][0]['astronomy'][0]['sunset']
        weather = beach_weather['data']['weather'][0]['hourly'][2]['weatherDesc'][0]['value']
        temp = beach_weather['data']['weather'][0]['hourly'][2]['tempF']
        watertemp = beach_weather['data']['weather'][0]['hourly'][2]['waterTemp_F']
        cloudcover = beach_weather['data']['weather'][0]['hourly'][2]['cloudcover']
        windspeed = beach_weather['data']['weather'][0]['hourly'][2]['windspeedMiles']
        swell = beach_weather['data']['weather'][0]['hourly'][2]['swellHeight_ft']
        
        # Append the city information into city_data list.
        beach_data.append({"Beach_Name": coordinates [i-1][0],
                           "Latitude": coordinates [i-1][1],
                           "Longitude": coordinates [i-1][2],
                          "Sunrise": sunrise,
                          "Sunset": sunset,
                          "Weather_Description": weather,
                          "Temperature": temp,
                          "Water_Temperature": watertemp,
                          "Cloud_Cover": cloudcover,
                          "Wind_Speed": windspeed,
                          "Wave_Height": swell
                         })

# If an error is experienced, skip the city.
    except:
        print("Beach information not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 --- Silver Strand State Beach
Processing Record 2 of Set 1 --- Imperial Beach
Processing Record 3 of Set 1 --- Border Field State Park
Processing Record 4 of Set 1 --- Pelican State Beach
Processing Record 5 of Set 1 --- Crescent Beach
Processing Record 6 of Set 1 --- Redwood National Park
Processing Record 7 of Set 1 --- Prairie Creek Redwoods State Beach
Processing Record 8 of Set 1 --- Humboldt Lagoons State Park
Processing Record 9 of Set 1 --- Sue-meg State Park
Processing Record 10 of Set 1 --- Trinidad State Beach
Processing Record 11 of Set 1 --- Little River State Beach
Processing Record 12 of Set 1 --- Clam Beach County Park
Processing Record 13 of Set 1 --- Sinkyone Wilderness State Park
Processing Record 14 of Set 1 --- Westport-Union Landing State Beach
Processing Record 15 of Set 1 --- MacKerricher State Park
Processing Record 16 of Set 1 --- Caspar Headlands State Beach
Processing Re

Beach information not found. Skipping...
Processing Record 12 of Set 5 --- Goleta Beach Park
Beach information not found. Skipping...
Processing Record 13 of Set 5 --- Arroyo Burro Beach
Beach information not found. Skipping...
Processing Record 14 of Set 5 --- Leadbetter Beach
Beach information not found. Skipping...
Processing Record 15 of Set 5 --- West Beach
Beach information not found. Skipping...
Processing Record 16 of Set 5 --- East Beach
Beach information not found. Skipping...
Processing Record 17 of Set 5 --- Butterfly Beach
Beach information not found. Skipping...
Processing Record 18 of Set 5 --- Lookout County Park
Beach information not found. Skipping...
Processing Record 19 of Set 5 --- Carpinteria State Beach
Beach information not found. Skipping...
Processing Record 20 of Set 5 --- Rincon Beach Park
Beach information not found. Skipping...
Processing Record 21 of Set 5 --- Emma Wood State Beach
Beach information not found. Skipping...
Processing Record 22 of Set 5 ---

In [37]:
#for i, beach in enumerate(coordinates):
#    beach_url = url + "&format=json&q=" + str(coordinates[i-1][1]) + "," + str(coordinates[i-1][2])
#    beach_weather = requests.get(beach_url).json()
    
#    sunrise = beach_data['data']['weather'][0]['astronomy'][0]['sunrise']
    
#    print(sunrise)

In [38]:
#convert dictionaries into a Pandas DF
beach_data_df = pd.DataFrame(beach_data)
beach_data_df.tail(20)

,Beach_Name,Latitude,Longitude,Sunrise,Sunset,Weather_Description,Temperature,Water_Temperature,Cloud_Cover,Wind_Speed,Wave_Height
193,San Elijo State Beach,33.022579,-117.284252,06:21 AM,04:47 PM,Clear,60,59,0,11,3.0
194,Cardiff State Beach,33.010506,-117.279630,06:21 AM,04:47 PM,Clear,60,59,0,11,3.0
195,Tide Beach Park,32.996866,-117.276454,06:21 AM,04:47 PM,Clear,60,59,0,11,3.0
196,Fletcher Cove Park,32.990891,-117.275296,06:21 AM,04:47 PM,Clear,60,59,0,11,3.0
197,Seascape Surf,32.985676,-117.272912,06:21 AM,04:47 PM,Clear,60,59,0,11,3.0
198,Del Mar City Beach,32.976959,-117.269502,06:21 AM,04:47 PM,Clear,60,59,0,11,3.0
199,Torrey Pines State Beach,32.937781,-117.261734,06:20 AM,04:47 PM,Clear,60,59,0,7,2.6
200,Black's Beach,32.891473,-117.255989,06:20 AM,04:47 PM,Clear,60,59,0,7,2.6
201,La Jolla Shores,32.885635,-117.253542,06:20 AM,04:47 PM,Clear,60,59,0,7,2.6
202,La Jolla Cove,32.850382,-117.273024,06:20 AM,04:47 PM,Clear,60,59,0,7,2.6


In [39]:
# Create the output file (CSV).
#output_data_file = "beaches_data3.csv"

# Export the Beach_Data into a CSV.
beach_data_df.to_csv(output_data_file, index_label="Beach_ID")